In [1]:
# Import Required Libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
from tqdm import tqdm

import os
import sys
import time
import math
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable

from scipy.ndimage import rotate
from pretrained_models.resnet20 import ResNetCIFAR

from functions import *
from importlib import reload

reload(sys.modules["functions"])

<module 'functions' from '/workspaces/Adversarial-Patches-Experimentation/10_code/functions.py'>

## Prepare Data

In [2]:
# PARAMS
batch_size = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# target = 1  # automobile

# CIFAR-10 image tensor mean and std
NORM_MEAN = [0.4914, 0.4822, 0.4465]
NORM_STD = [0.2023, 0.1994, 0.2010]

# TODO: should we apply the below transformations?
transform_train = transforms.Compose(
    [
        # transforms.RandomCrop(32, padding=4),
        # transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean=NORM_MEAN, std=NORM_STD),
    ]
)

transform_test = transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)]
)

transform_image = transforms.ToPILImage()

print("==> Preparing data..")
trainset = torchvision.datasets.CIFAR10(
    root="../00_data", train=True, download=True, transform=transform_train
)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=batch_size, shuffle=True, num_workers=4
)

testset = torchvision.datasets.CIFAR10(
    root="../00_data", train=False, download=True, transform=transform_test
)
testloader = torch.utils.data.DataLoader(
    testset, batch_size=100, shuffle=False, num_workers=2
)

==> Preparing data..
Files already downloaded and verified
Files already downloaded and verified


In [3]:
# Train-Validation Split
train_set, val_set = torch.utils.data.random_split(trainset, [0.8, 0.2])
train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=4
)
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=32, shuffle=False, drop_last=False, num_workers=4
)

## Run the Model with the Patch Attack

In [4]:
# load the Pre-Trained ResNet-20 model
net = ResNetCIFAR(num_layers=20, Nbits=None)
net = net.to(device)
net.load_state_dict(torch.load("./pretrained_models/pretrained_model.pt"))

<All keys matched successfully>

### Circle Patch

In [6]:
p, x = patch_attack_untargeted(net, device, train_loader, val_loader, patch_size=16, patch_type="circle", num_epochs=20)

Epoch 0, Attack Success Rate: 0.7618749737739563.


Epoch 5, Attack Success Rate: 0.8999999761581421.


Epoch 10, Attack Success Rate: 0.8999999761581421.


Epoch 15, Attack Success Rate: 0.8999999761581421.


Epoch 19, Attack Success Rate: 0.8999999761581421.


### Square Patch

In [7]:
p_sq, x_sq = patch_attack_untargeted(net, device, train_loader, val_loader, patch_size=16, patch_type="square", num_epochs=20)

Epoch 0, Attack Success Rate: 0.9036999940872192.


Epoch 5, Attack Success Rate: 0.9036999940872192.


Epoch 10, Attack Success Rate: 0.9036999940872192.


Epoch 15, Attack Success Rate: 0.9036999940872192.


Epoch 19, Attack Success Rate: 0.9036999940872192.
